In [1]:
import numpy as np
import torch
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


We generate the data for the Lorenz system using the Euler method.

In [3]:
import numpy as np

#Uses Euler method to solve system of differential equations for Lorenz system

def Lorenz(T, dt, N_sims,s,r,b):
    N_t  = int(T//dt)
    sims = np.zeros((N_sims, N_t, 3))
    for sim in range(N_sims):
        sims[sim,0,0] = 0.
        sims[sim,0,1] = 1.
        sims[sim,0,2] = 1.05
        for i in range(1,N_t):
            sims[sim, i, 0] = sims[sim,i-1,0] + dt*(s*(sims[sim,i-1,1]-sims[sim,i-1,0]))
            sims[sim, i, 1] = sims[sim,i-1,1] + dt*(r*(sims[sim,i-1,0])-sims[sim,i-1,1] - (sims[sim,i-1,0]*sims[sim,i-1,2]))
            sims[sim, i, 2] = sims[sim,i-1,2] + dt*( -b*(sims[sim,i-1,2]) + (sims[sim,i-1,0]*sims[sim,i-1,1]))

    return sims.astype(np.float32)




def prepare_data_fast(data_x, delay, normalize, irregular_delays = None):
  #Normalize data and take its transpose
    data = data_x.T / normalize

  # Y = (Y_1,..,Y_N) where Y_k= x_(k+1)

    Y = data[delay:]

    #interleaves time differences to X
    if irregular_delays is not None:
        data = np.concatenate((data,irregular_delays[...,None]),-1)

  # create array X = (X_1,...X_N) where X_k = (x_k,...,x_k-tau+1)
    #X = np.zeros((data.shape[0]-delay,delay*data.shape[1]))
    X = np.zeros((data.shape[0]-delay,delay))
    for k in range(X.shape[0]):
      #X_k = (x_k,...,x_{k+delay-1})
        X[k] = data[k:(k+delay)].reshape(-1)

    return X, Y

We define our kernel

In [5]:
import torch
import numpy as np



# Returns the norm of the pairwise difference
def norm_matrix(matrix_1, matrix_2):
  #Given X = (X_1, ..., X_N), Y = (Y_1, ..., Y_N) output matrix M with entries M_{ij} = ||X_i-Y_j||**2'
  #If X and Y matrices then X_1,...X_N and Y_1,...,Y_N are rows of X and Y and M returns squared Euclidean distance between rows of X and Y
  #If X and Y are vectors, M returns squared difference between entries of X and Y
    #torch.square is the elementwise square of array e.g. torch.square([[2,3],[3,4]]) = [[4,9],[9,16]]
    #torch.sum(A,axis=1) is vector containing row sum of matrix elements
    #torch.reshape(A,(-1,1)) turns each element of the array into its own block torch.reshape([[1,2]])= [[1],[2]]
    norm_square_1 = torch.sum(torch.square(matrix_1), axis = 1)
    norm_square_1 = torch.reshape(norm_square_1, (-1,1))

    norm_square_2 = torch.sum(torch.square(matrix_2), axis = 1)
    norm_square_2 = torch.reshape(norm_square_2, (-1,1))

    #matrix_1.shape is tuple containing number of rows and columns of matrix

    d1=matrix_1.shape
    d2=matrix_2.shape

    #if d1 and d2 have different number of columns, take transpose of matrix_1

    if d1[1]!=d2[1]:
        matrix_1=torch.transpose(matrix_1)

    #torch.transpose(matrix_2,0,1) is just usual matrix transpose
    #torch.matmul(A,B) is normal matrix multiplication

    inner_matrix = torch.matmul(matrix_1, torch.transpose(matrix_2,0,1))

    #inner_matrix entries are sum_i sum_j (A_ij B_ji)
    # norm_diff gives us -2 sum_i sum_j (A_ij B_ji)  sum_j A_ij^2 + sum_i B_ij^2

    norm_diff = -2 * inner_matrix + norm_square_1 + torch.transpose(norm_square_2,0,1)

    return norm_diff



# Returns the pairwise inner product
#e.g. Let X=(X_1,X_2,...,X_N) and Y= (Y_1,Y_2,..,Y_N) be matrices where X_i and Y_i are the rows of X and Y
#then the entries of C=inner_matrix(X,Y) are C_ij = np.dot(X_i,Y_j), the dot product of X_i and Y_j
def inner_matrix(matrix_1, matrix_2):

    d1=matrix_1.shape
    d2=matrix_2.shape
    #if d1 and d2 have different number of columns, take transpose of matrix_1

    if d1[1]!=d2[1]:
        matrix_1=torch.transpose(matrix_1,0,1)
    return torch.matmul(matrix_1, torch.transpose(matrix_2,0,1))

def kernel_anl3(matrix_1, matrix_2, parameters, coefs):
    i=0
    j=0


    matrix = norm_matrix(matrix_1, matrix_2)
    matrix[[matrix<0]] = 0
    sigma = parameters[i+0]
    #torch.exp(matrix)- matrix that returns exponential of each matrix element
    K =  torch.exp(-matrix/ (2* sigma**2))
    K=K*(coefs[j+0])**2
    i=i+1
    j=j+1


    c = (parameters[i])**2
    imatrix = inner_matrix(matrix_1, matrix_2)
    K = K+ (coefs[j])**2 *(imatrix+c) ** 2

    i=i+1
    j=j+1

    beta = parameters[i]
    gamma = (parameters[i+1])**2
    K=K+ (coefs[j])**2 *(beta**2 + gamma*matrix)**(-1/2)

    i=i+2
    j=j+1

    alpha = parameters[i]
    beta = parameters[i+1]
    K=K+ (coefs[j])**2 *(beta**2 + matrix)**(-alpha)

    i=i+2
    j=j+1

    sigma = parameters[i]
    K=K+ (coefs[j])**2 * 1/(1 + matrix/sigma**2)

    i=i+1
    j=j+1

    alpha_0 = parameters[i]
    sigma_0 = parameters[i+1]
    alpha_1 = parameters[i+2]
    sigma_1 = parameters[i+3]

    K =  K+ (coefs[j])**2 *alpha_0*torch.maximum(torch.zeros(1,device = parameters.device), 1-matrix/(sigma_0))+ alpha_1 * torch.exp(-matrix/ (2* sigma_1**2))
    i=i+4
    j=j+1

    p = parameters[i]
    l = parameters[i+1]
    sigma = parameters[i+2]
    K =K+ (coefs[j])**2 * torch.exp(-torch.sin(matrix*np.pi/p)**2/l**2)*torch.exp(-matrix/sigma**2)

    i=i+3

    p = parameters[i]
    l = parameters[i+1]
    K = K+ (coefs[j])**2 *torch.exp(-torch.sin(matrix*np.pi/p)/l**2)

    i=i+2
    j=j+1


    return K

def kernel_matern(matrix_1,matrix_2, parameters):
  sigma = parameters[0]
  l = parameters[1]
  matrix = norm_matrix(matrix_1,matrix_2)
  matrix[[matrix<0]] = 0

  K = sigma**2*(1+ np.sqrt(5)*torch.sqrt(matrix)/l+5*matrix/(3*l**2))*torch.exp(-np.sqrt(5)*torch.sqrt(matrix)/l)

  return torch.Tensor(K)

def kernel_linear(matrix_1,matrix_2,parameters):
  sigma = parameters[0]
  c = parameters[1]
  matrix = inner_matrix(matrix_1,matrix_2)

  K = sigma**2*(matrix+c)

  return torch.Tensor(K)


In [6]:
def Big_kernel(matrix_1, matrix_2, parameters, coefs):

  norm = norm_matrix(matrix_1,matrix_2)
  norm[[norm<0]] = 0
  inner = inner_matrix(matrix_1,matrix_2)
  M = torch.zeros(norm.shape)
  M[[norm<parameters[33]**2]] = 1
  K = coefs[0]**2*(inner+parameters[0]**2)
  K +=  coefs[1]**2*torch.pow(parameters[1]**2*inner+parameters[2]**2,torch.abs(parameters[3]))
  K += coefs[2]**2*torch.exp(-norm/(2*parameters[4]**2))
  K += coefs[3]**2*torch.exp(-torch.sqrt(norm)/(2*parameters[5]**2))
  K += coefs[4]**2*torch.exp(-torch.sin(np.pi*norm/parameters[6])**2/parameters[7]**2)*torch.exp(-norm/parameters[8]**2)
  K += coefs[5]**2*torch.exp(-torch.sin(np.pi*norm/parameters[9])**2/parameters[10]**2)
  K += coefs[6]**2*torch.exp(-torch.sin(np.pi*torch.sqrt(norm)/parameters[11])**2/parameters[12]**2)*torch.exp(-torch.sqrt(norm)/parameters[13]**2)
  K += coefs[7]**2*torch.exp(-torch.sin(np.pi*torch.sqrt(norm)/parameters[14])**2/parameters[15]**2)
  K += coefs[8]**2*torch.sqrt(norm+parameters[16]**2)
  K += coefs[9]**2*torch.pow(parameters[17]**2+parameters[18]**2*norm,-1/2)
  K += coefs[10]**2*torch.pow(parameters[19]**2+parameters[20]**2*torch.sqrt(norm),-1/2)
  K += coefs[11]**2*torch.pow(parameters[21]**2+torch.sqrt(norm),parameters[22])
  K += coefs[12]**2*torch.pow(parameters[23]**2+norm,parameters[24])
  K += coefs[13]**2*torch.pow(1+norm/parameters[25],-1)
  K = K + coefs[14]**2*(1/(1+torch.sqrt(norm)/parameters[26]**2))
  K = K + coefs[15]**2*(1-(1-norm/(norm+parameters[27]**2)))
  K = K + coefs[16]**2*torch.maximum(torch.zeros(norm.shape),1-torch.sqrt(norm)/parameters[28]**2)
  K = K + coefs[17]**2*torch.maximum(torch.zeros(norm.shape),1-norm/parameters[29]**2)
  K = K + coefs[18]**2*(torch.log(torch.sqrt(norm)**parameters[30]+1))
  K = K + coefs[19]**2*(torch.tanh(parameters[31]*inner+parameters[32]))
  #K = K + coefs[20]**2*(torch.arccos(-torch.sqrt(norm)/parameters[33]**2)-(torch.sqrt(norm)/parameters[33]*torch.sqrt(1-(torch.sqrt(norm)/parameters[33]**2)**2)))*M
  return(K)

In [7]:
import math

def sample_selection(N, size):
  #indices: vectors containing integers from 0 to N-1 inclusive
    indices = np.arange(N)
  #sample_indices: takes sample of indices (without replacement) and arranges them in ascending order
    sample_indices = np.sort(np.random.choice(indices, size, replace= False))
    return sample_indices

# The pi or selection matrix
def pi_matrix(sample_indices, dimension):
  #torch.zeros creates tensor of zeros
  #dimension is a tuple that contains the number of rows and cols of matrix pi
  #.double() converts tensor to float64
    pi = torch.zeros(dimension).double()

  #In the ith row of pi, replace entries corresponding to ith entries of 'sample_indices' with 1:

  #e.g. pi = [[0,0,0,0],[0,0,0,0],[0,0,0,0]] and sample_indices = [0,2,3]
  #then pi becomes [[1,0,0,0],[0,0,1,0],[0,0,0,1]]

    for i in range(dimension[0]):
        pi[i][sample_indices[i]] = 1

    return pi

def batch_creation(N, batch_size, sample_proportion = 0.5):
    #batch_size ==False means we use the full training data in the batch when computing the kernel matrix
    if batch_size == False:
        batch_indices = np.arange(N)
    #if batch_size is between 0 and 1 (that is, a proportion), we use (N*batch_size) % of the training data in our batch
    elif 0 < batch_size <= 1:
        batch_size = int(N * batch_size)
        #returns indices of batch
        batch_indices = sample_selection(N, batch_size)
    else:
    #if batch_size is an integer, we use that number as the number of samples in our batch
        batch_indices = sample_selection(N, batch_size)

    # Sample from the mini-batch (sample of the sample)
    #we use (batch_size*sample_proportion)% of the batch in our mini-batch
    sample_size = math.ceil(len(batch_indices)*sample_proportion)
    sample_indices = sample_selection(len(batch_indices), sample_size)

    return sample_indices, batch_indices

class KernelFlows(torch.nn.Module):

    def __init__(self, kernel_keyword, nparameters, nfamilies, regu_lambda, regu_lambda2, dim, metric = "rho_ratio", batch_size = 100):
        super().__init__()

        #init gives us definitions we will be using in the rest of the code in class KernelFlows

        #e.g. 'rbf', 'kernel_anl3', states the type of kernel we use
        self.kernel_keyword = kernel_keyword

        self.regu_lambda = regu_lambda

        self.regu_lambda2 = regu_lambda2

        #we use a vector of ones as our initial guess for the parameters before we learn them
        self.kernel_params = torch.nn.Parameter(torch.ones(nparameters),requires_grad = True)
        self.coefs = torch.nn.Parameter(torch.ones(nfamilies),requires_grad = True)

        #we use the kernel_anl3
        self.kernel = kernel_anl3                                ######################## CHOIX DU KERNEL ########################

        #dim is dimension of dynamical system
        self.dim = dim

        #batch_size is the number of samples we use in our batch at each iteration when computing kernel matrix K(X_beta, X_beta) in KF algorithm
        self.batch_size = batch_size

        #type of rho metric we use; we use metric='rho ratio' in our paper

        if metric == "rho_ratio":
            self.rho_fun = self.rho_ratio
        elif metric == "rho_general":
            self.rho_fun = self.rho_general
        else:
            raise("Metric not supported")
        self.metric = metric


    def set_training_data(self,X,Y):
      #gives us X_train and Y_train
        self.X_train = X
        self.Y_train = Y


    def rho_ratio(self, matrix_data, Y_data, sample_indices,  regu_lambda = 0.000001, regu_lambda2 = 0.000001):
        #use CPU for faster matrix computations
        self.device = torch.device("cpu")

        #K(X,X)

        kernel_matrix = self.kernel(matrix_data, matrix_data, self.kernel_params, self.coefs)
        #pi_matrix is Boolean matrix..float()
        pi = pi_matrix(sample_indices, (sample_indices.shape[0], matrix_data.shape[0])).to(matrix_data.device)

        #sample_matrix = K(X_beta,X_beta) (kernel matrix in the denominator of the kernel flows algorithm)
        #basically we discard entries of K(X_pi,X_pi) that aren't in the sub-sample and create a new, smaller, matrix K(X_beta,X_beta)
        sample_matrix = torch.matmul(pi, torch.matmul(kernel_matrix, torch.transpose(pi,0,1)))

        #Y_beta
        Y_sample = Y_data[sample_indices]

        #(K(X_pi,X_pi)+lambda*I)^-1
        inverse_data = torch.linalg.inv(kernel_matrix + regu_lambda * torch.eye(kernel_matrix.shape[0], device = matrix_data.device))
        #(K(X_pi,X_pi)+lambda*I)^-1
        inverse_sample = torch.linalg.inv(sample_matrix + regu_lambda * torch.eye(sample_matrix.shape[0], device = self.device))
        #Y_beta^T (K(X_beta,X_beta)+lambda*I)^-1 Y_beta
        Y_data = torch.reshape(Y_data,(matrix_data.shape[0],1))
        Y_sample = torch.reshape(Y_sample,(len(sample_indices),1))

        top = torch.tensordot(Y_sample, torch.matmul(inverse_sample, Y_sample))
        #Y_pi^T (K(X_pi,X_pi)+lambda*I)^-1 Y_pi
        bottom = torch.tensordot(Y_data, torch.matmul(inverse_data, Y_data))

        # rho = 1 - (Y_beta^T (K(X_beta,X_beta)+lambda*I)^-1 Y_beta)/(Y_pi^T (K(X_pi,X_pi)+lambda*I)^-1 Y_pi)
        rho = 1 - top/bottom + self.regu_lambda2*torch.norm(self.coefs,p=1)
        #rho = 1 - top/bottom
        return rho

    def rho_general(self, matrix_data, Y_data,  regu_lambda = 0.000001, **kwargs):
        #rho_general is another metric for rho, this time we don't take any sub-batches

        #compute K(X,X)

        kernel_matrix = self.kernel(matrix_data, matrix_data, self.kernel_params, self.coefs)
        #normalize kernel matrix by dividing by the trace

        kernel_matrix = kernel_matrix / torch.trace(kernel_matrix)

        #compute K(X,X)+lambda*I

        inverse_matrix = torch.linalg.inv(kernel_matrix + regu_lambda * torch.eye(kernel_matrix.shape[0]))

        #then rho= Y^T  (K(X,X)+lambda*I) Y

        rho = torch.tensordot(Y_data, torch.matmul(inverse_matrix, Y_data))

        return rho

    def forward(self, adaptive_size = False, proportion = 0.5):

        if adaptive_size == False or adaptive_size == "Dynamic":
            sample_size = proportion
        else:
            print("Sample size not recognized")


        # Create a batch and a sample
        sample_indices, batch_indices = batch_creation(self.X_train.shape[0], batch_size= self.batch_size, sample_proportion = sample_size)
        X_data = self.X_train[batch_indices]
        Y_data = self.Y_train[batch_indices]

        #optimizer and backward

        #calculates rho

        rho = self.rho_fun( X_data, Y_data,
                                       sample_indices = sample_indices, regu_lambda = self.regu_lambda, regu_lambda2 = self.regu_lambda2)

        return rho


    def get_parameters(self):
      #returns kernel parameters
        return self.kernel_params

    def get_coefs(self):
      #returns kernel parameters
        return self.coefs

    def compute_kernel_and_inverse(self,regu_lambda = 0.0000001):
        X_data = self.X_train
        #computes kernel matrix K(X,X) from training data
        self.kernel_matrix = self.kernel(X_data,X_data, self.kernel_params, self.coefs)
        #adds small nugget term lambda
        self.kernel_matrix += regu_lambda * torch.eye(self.kernel_matrix.shape[0], device = X_data.device)
        #computes inverse of K+lambda*I
        self.inverse_kernel = torch.linalg.inv(self.kernel_matrix)
        #computes (K+lambda*I)^-1 Y
        self.A_matrix = torch.matmul(self.inverse_kernel,self.Y_train)


    def predict(self,x_test):
        #kernel_pred is vector K(x,X)
        kernel_pred = self.kernel(x_test,self.X_train,self.kernel_params,self.coefs)
        #uses representer formula K(x,X)(K+lambda*I)^-1 Y to make prediction
        prediction = torch.matmul(kernel_pred,self.A_matrix)
        return prediction


    def predict_ahead(self,x_test, horizon, delay, delta_t_mode = False, device = torch.device("cpu")):
        """
        Perform n=horizon steps ahead prediction.
        If delta_t_mode is True, x_test is expected to have the the following structure (X(t-1),delta_t-1,X(t),delta_t))

        out_dim is the dimension of the y vector (and of the observations in x as well)
        delay : delay used in the x
        """
        assert horizon >0 # minimum horizon is 1
        assert delay >0

        device = torch.device("cpu")

        #initialize Y_p, the vector that contains our predictions

        Y_p = torch.zeros((x_test.shape[0],self.dim))

        #converts test data to Pytorch tensor,.double() converts to float, to.device() means we can use GPU
        X_test_ = torch.Tensor(x_test).double().to(device)

        if delta_t_mode:
            indices_delays = [((self.dim+1)*i,(self.dim+1)*i+1) for i in range(delay)] # We should not touch the delta t
        else:
            indices_delays = [(self.dim*i,self.dim*i+1) for i in range(delay)]

        # Make sure there is no contamination (deleting the previous values)
        for dim in range(horizon):
            n_delays = min(dim,delay)
            #for n in range(1,n_delays+1):
             #   X_test_[dim::horizon][:,indices_delays[-n]] = 0

        # Predicting in chunks and propagating predictions to the next step.
        for dim in range(horizon):
          #Predict Y_dim, Y_(dim+horizon), Y_(dim+2*horizon) etc. using representer formula
          #then update X__(dim+1),..., X_(dim+horizon-1)
          #then predict Y_(dim+1), Y_(dim+horizon+1), Y_(dim+2*horizon+1),...,

            #Y_p[dim::horizon] = self.predict(X_test_[dim::horizon])
            Y_p[dim::horizon] = torch.reshape(self.predict(X_test_[dim::horizon]),Y_p[dim::horizon].shape)
            for dim_plus in range(dim+1,min(horizon,delay+dim+1)):
                l_x = X_test_[dim_plus::horizon].shape[0]
                idx = dim_plus-dim
                #now set X_(dim+1) = Y_dim and use that to predict Y_(dim+1) at next iteration etc.
                X_test_[dim_plus::horizon][:,indices_delays[-1*idx][0]:2+indices_delays[-1*idx][1]] = Y_p[dim::horizon][:l_x] # should be -1-2:-1 for irregular

        return Y_p




In [8]:
def train_kernel(X_train, Y_train, model,  lr = 0.1, epochs=1000,verbose= False):
    """
    dim is the dimension of a single observation
    """
    #the next line states that we are optimizing the kernel parameters using SGD algorithm with learning rate lr
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)

    #states that we use X_train and Y_train as our training data
    #.double() converts tensor elements to float64

    model.set_training_data(torch.Tensor(X_train).double(),torch.Tensor(Y_train).double())

    #tqdm showa progress bar

    for i in tqdm.tqdm(range(epochs)):
      #sets gradients of all optimized torch.Tensors to 0
        optimizer.zero_grad()
        #computes rho
        rho = model.forward()
        #rho must lie between 0 and 1 inclusive otherwise by definition
        if rho>=0 and rho<=1 and model.metric=="rho_ratio":
          #rho.backward() computes gradient of rho by back-propagation
            rho.backward()
          #optimizer.step() updates parameter values at each step
            optimizer.step()
            model.parameters()

          #verbose =True would print rho at each epoch
            if verbose:
                print(rho)

    return model

In [9]:
def replace_nan_last(array):
    #array=array.reshape(1,len(array))
    if (array[0]=='NA'):
        array[0]=0
    for i in range(1,len(array)):
        if (array[i]=='NA'):
            array[i] = array[i-1]
    return array

## Data Generation

In [10]:
import json
with open('/content/drive/MyDrive/DATA/tsdl.JSON', 'r', encoding='cp1252') as f:    ## YOU HAVE TO CHANGE THIS LINE
    data = json.load(f)

donnees = []
def ts_multi(ind):

    error_metrics = []
    Data = data[ind]['values']

    if data[ind]['type']=='univariate':
        Data=replace_nan_last(Data)
    else:
        Data=[replace_nan_last(Data[i]) for i in range(len(Data)) ]

    Data=np.array(Data)
    if data[ind]['type']=='univariate':
        Data = Data.astype(np.float32)
        Data.reshape((len(Data),1))
    max_delay = 1
    #total samples we use in training+testing
    N_points = len(Data)
    train_n  = int(len(Data)*0.7)
    burnin = 0
    dt = 0.01
    dt = 1

    #time index differences betweeen observations
    #create random vector of length N_points-1 where each element lies between 1 and N
    delays = np.random.randint(max_delay,size=N_points-1)+1
    #observed time indices
    #we add np.zeros(1) to indicate that we observe the initial state at time t=0
    indices = np.concatenate((np.zeros(1),np.cumsum(delays))).astype(int)
    #we add np.zeros(1) to ensure that delays and indices are of the same length
    delays = np.concatenate((delays,np.zeros(1))).astype(int)

    #final time index
    max_idx = indices[-1]
    #final time= final time index*step size (dt)
    #T = np.ceil(max_idx*dt)

    observed_data = Data[indices]

    #Use first train_n points of (irregularly sampled) data as training set
    train_data = observed_data[:train_n].T

    #Use remaining points as test data
    test_data = observed_data[train_n:].T
    #time indices for train data
    delays_train = delays[:train_n]
    #time indices for test data
    delays_test = delays[train_n:]




##### Apply kernel flow from the data

    # Some constants
    #nparameters=34    ## ANOTHER KERNEL WITH nfamillies = 20
    nparameters=26
    #delay is tau in the paper (number of samples in X we use)
    delay = 10

    #nugget term
    regu_lambda = 0.001
    regu_lambda2=0.0001
    #learning rate
    lr = 0.01

    # Get scaling factor
    normalize=np.amax(train_data)

    #gives us X_train, Y_train as defined in paper
    X_train, Y_train = prepare_data_fast(train_data,delay,normalize)
    #gives us X_test, Y_test
    X_test, Y_test = prepare_data_fast(test_data,delay,normalize)
    dim = 1
    #defines the model we are using, family of kernel, number of parameters, nugget term lambda, dimension of dynamical system, optimization metric rho, batch size
    model = KernelFlows("anl3", nparameters = nparameters, regu_lambda = regu_lambda, regu_lambda2=regu_lambda2, nfamilies= 8 , dim = dim, metric = "rho_ratio", batch_size = 50) #nfamilies=20
    #now we train the kernel using this kernel flows model model
    model  = train_kernel(X_train, Y_train, model, lr = lr, epochs = 1000)

    #computes kernel matrix from learned parameters and inverse (add nugget regu_lambda to ensure numerical stability)
    model.compute_kernel_and_inverse(regu_lambda = regu_lambda)
    print(model.compute_kernel_and_inverse())

    horizon = 2
#make predictions within a 20 time step horizon
#delta_t_mode=False means we are not incorporating time differences in kernel

    Y_pred = model.predict_ahead(X_test,horizon=horizon, delay = delay, delta_t_mode = False)

#Mean squared error (mean squared distance from the true value) and R2 statistic (between 0 and 1, measures how well data fits model, )
    mse_pred = (Y_pred.detach()-Y_test).pow(2).mean()
    r2 = 1-mse_pred/(Y_test.var())
    print(f"MSE On test : {mse_pred:.4f} - R2 : {r2:.4f}")
# SMAPE
    numerator = np.abs(Y_pred.detach() - Y_test)
    denominator = (np.abs(Y_pred.detach()) + np.abs(Y_test)) / 2
    smape = torch.mean(numerator / denominator) *100
    print(f"SMAPE On test : {smape:.4f}")

    def custom_format(value):
      if np.isnan(value) or np.isinf(value):
        return "$<$"
      elif abs(value) > 999:
        return "$>>1$"
      else:
        return f"{value:.4f}"

    def custom_format_r2(value):
      if np.isnan(value) or np.isinf(value):
        return "$<$"
      elif abs(value) > 1:
        return "$<$"
      else:
        return f"{abs(value):.4f}"
    #donnees.append([ind, f'{mse_pred:.4f}', f'{r2:.4f}', f'{smape:.4f}'])
    donnees.append([ind, custom_format(mse_pred), custom_format_r2(r2),custom_format(smape)])
    print(model.get_coefs())
    print(model.get_parameters())

    from pylab import rcParams
    from IPython.display import display

#Plot predicted time series of x dimension versus truth
    if dim == 1 :
      """fig, ax = plt.subplots(dim,1,figsize=(16,9))
      ax.plot(normalize*Y_pred.detach(),label = "pred")
      ax.plot(normalize*Y_test, label = "true")
      ax.set_xlabel('Observation')
      ax.set_ylabel('x')
      ax.set_title(f"{data[ind]['description']}. Index : {ind}. Taille : {len(data[ind]['values'])}")
      ax.legend(loc = "upper right")
      print(f"MSE On test : {mse_pred:.4f} - R2 : {r2:.4f}")"""
      fig, ax = plt.subplots(dim, 1, figsize=(12, 6 * dim))

# Plot the prediction and true values
      ax.plot(normalize * Y_pred.detach(), label="Prediction", color='green')
      ax.plot(normalize * Y_test, label="True", color='blue')   #Y_test



# Customize plot appearance
      ax.set_xlabel('Observation')
      ax.set_ylabel('Value')
      ax.set_title(f"{data[ind]['description']} - Index: {ind} - Size: {len(data[ind]['values'])}")
      ax.legend(loc="upper left")
      ax.grid(True)
      x_ticks = range(0, len(Y_test), 10)  # Adjust the interval as needed    #Y_test
      ax.set_xticks(x_ticks)
      ax.set_xticklabels(x_ticks, rotation=45, ha='right')  # Adjust x-axis tick positions

# Format x-axis date labels if dealing with dates (example)
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Adjust layout to prevent overlap of labels
      plt.tight_layout()

# Show the plot
      plt.show()

# Print evaluation metrics
      #print(f"MSE on test: {mse_pred:.4f} - R2: {r2:.4f}")
    else :
      fig, ax = plt.subplots(dim,1,figsize=(6.4,8.5))
      ax[0].plot(normalize*Y_pred.detach(),label = "pred")
      ax[0].plot(normalize*Y_test, label = "true")
      ax[0].set_xlabel('Observation')
      ax[0].set_ylabel('x')
      ax[0].set_title("First Dimension")
      ax[0].legend(loc = "upper right")
      display(fig)
      print(f"MSE On test : {mse_pred:.4f} - R2 : {r2:.4f}")

In [14]:
L_det = [0, 2, 3, 84, 106, 118, 119, 121, 122, 126, 131, 149, 160, 186, 190, 193, 198, 199, 203, 227, 244, 246, 328, 382, 390, 463, 534, 543, 578, 608, 643]
L_sto = [4, 6, 9, 78, 105, 112, 113, 130, 136, 145, 158, 171, 212, 215, 218, 221, 231, 232, 248, 254, 260, 263, 273, 282, 296, 314, 338, 352, 547, 615]
L_statio =  [89, 90, 91, 171, 195, 208, 214, 217, 227, 229, 230, 231, 232, 247, 254, 273, 293, 314, 315, 338, 352, 375, 452, 502, 539, 547, 610, 612, 615, 623]
L_non_statio = [0, 3, 5, 16, 17, 18, 61, 95, 96, 98, 99, 106, 121, 122, 123, 131, 132, 149, 160, 193, 202, 203, 246, 251, 382, 390, 391, 400, 478, 483]

import pandas as pd

# Créez une liste vide pour stocker les données

donnees = []


for ind in L_sto:
  ts_multi(ind)
# Créez un DataFrame à partir des données
#table = pd.DataFrame(donnees, columns=['Index', 'MSE', 'R^2', 'SMAPE'])

# Affichez la table
#print(table)
#table.to_csv('table_L_non_statio.csv', index=False)

 48%|████▊     | 482/1000 [00:03<00:03, 154.36it/s]


KeyboardInterrupt: ignored